# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

f2cb607558014024becb02a9ef195954


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,langsa,4.4683,97.9683,29.77,59,86,0.98,ID,1676516361
1,1,bengkulu,-3.8004,102.2655,27.70,68,98,0.84,ID,1676516361
2,2,vaini,-21.2000,-175.2000,29.09,84,75,5.14,TO,1676516361
3,3,vysokogornyy,50.1167,139.1500,-16.25,81,90,2.73,RU,1676516362
4,4,marzuq,14.4000,46.4667,14.51,15,5,2.59,YE,1676516362


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    scale = .5,
    tiles = "OSM"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df['Max Temp'] <26) & (city_data_df['Max Temp'] > 21)]
ideal_city

# Drop any rows with null values
ideal_city = ideal_city.dropna(how='any')

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,te anau,-45.4167,167.7167,22.15,38,69,1.64,NZ,1676516345
7,7,saint-pierre,-21.3393,55.4781,24.39,68,0,3.60,RE,1676516363
9,9,champerico,14.3000,-91.9167,25.28,77,0,2.02,GT,1676516364
11,11,pisco,-13.7000,-76.2167,23.03,94,20,2.06,PE,1676516365
14,14,cidreira,-30.1811,-50.2056,23.40,89,29,3.13,BR,1676516188
...,...,...,...,...,...,...,...,...,...,...
547,547,sambava,-14.2667,50.1667,23.40,91,15,2.17,MG,1676516576
559,559,alindao,5.0267,21.2088,21.95,31,100,2.05,CF,1676516597
560,560,dondo,-19.6094,34.7431,23.73,95,3,2.25,MZ,1676516597
561,561,bhainsdehi,21.6467,77.6325,21.80,23,0,1.44,IN,1676516598


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,te anau,NZ,-45.4167,167.7167,38,
7,saint-pierre,RE,-21.3393,55.4781,68,
9,champerico,GT,14.3000,-91.9167,77,
11,pisco,PE,-13.7000,-76.2167,94,
14,cidreira,BR,-30.1811,-50.2056,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'

# filters = f'circle:{lng},{lat},{radius}'
# bias = f'proximity:{lng},{lat}'
limit = 1 

params = {
    "categories":categories,
    "limit":limit,
    # "filter": filters,
    # "bias": bias,
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
  
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
te anau - nearest hotel: Lakeside Motels & Apartments
saint-pierre - nearest hotel: Tropic Hotel
champerico - nearest hotel: Hotel y Restaurante El Submarino
pisco - nearest hotel: La Portada
cidreira - nearest hotel: Hotel Castelo
busselton - nearest hotel: Broadwater Beach Resort
san felipe - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
saint-philippe - nearest hotel: Le Baril
alugan - nearest hotel: RJV hôtel
east london - nearest hotel: No hotel found
bathsheba - nearest hotel: Atlantis Hotel
najran - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
arraial do cabo - nearest hotel: No hotel found
kuliyapitiya - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
paranaiba - nearest hotel: Hotel Sul
bambous virieux - nearest hotel: Otentic Eco Tent Experience
adrar - nearest hotel: No hotel found
sao filipe - nearest hotel: Bela Vista
tenkasi - nearest hotel: Murugan Mahal
santa maria - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
5,te anau,NZ,-45.4167,167.7167,38,Lakeside Motels & Apartments
7,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
9,champerico,GT,14.3000,-91.9167,77,Hotel y Restaurante El Submarino
11,pisco,PE,-13.7000,-76.2167,94,La Portada
14,cidreira,BR,-30.1811,-50.2056,89,Hotel Castelo
...,...,...,...,...,...,...
547,sambava,MG,-14.2667,50.1667,91,Royal Sava Hôtel
559,alindao,CF,5.0267,21.2088,31,No hotel found
560,dondo,MZ,-19.6094,34.7431,95,No hotel found
561,bhainsdehi,IN,21.6467,77.6325,23,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    scale = 1,
    tiles = "OSM",
    hover_cols = ['Hotel Name', 'Country']
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)